In [62]:
import pandas as pd
import matplotlib

%matplotlib inline

GBQ_PROJECT_ID = '620265099307'
# This appears to be the "FreeStyle Libre Sensor" only (i.e. not the full kit), of which you need 2 per month

libre_code = "21480000100"  
test_strip_code = "0601060D0%"
type_one_in_england = 311640 # see below


> It is estimated that between 3% and 5% of patients with type 1 diabetes in England have access to Freestyle Libre, but if all CCGs followed the guidance correctly this figure could rise to around 25%

and

> Official prescribing data collated by the diabetes campaigner Nick Cahm and shared with The BMJ indicate that only 2% of patients in England with type 1 diabetes are getting FreeStyle Libre on GP prescription, a much smaller proportion than the 11% in Scotland, 16% in Wales, and 35% in Northern Ireland

## Background data

Approximately 400,000 people are currently living with type 1 diabetes in the UK, with over 29,000 of them children [1](https://jdrf.org.uk/information-support/about-type-1-diabetes/facts-and-figures/)

|Prevalence|	2016 - 2017|
|---|---|
|England	| 3,116,399|
|Northern Ireland |	92,480|
|Scotland|	289,040|
|Wales|	191,590|
|UK|	3,689,509|
[2](https://www.diabetes.org.uk/professionals/position-statements-reports/statistics/diabetes-prevalence-2017)


Around 90% of people have Type 2 diabetes. Around 10% of people have Type 1 diabetes. [3](https://www.gov.uk/government/news/38-million-people-in-england-now-have-diabetes)

That gives us approx 311,640 people with Type 1.

The 2015 NICE guidelines recommend that people with type 1 diabetes test their blood glucose at least 4 times per day

For Type 2, the recommendation is:

> Self-monitoring isn't suitable for everyone with type 2 diabetes, and you should only self-monitor if your doctor or nurse advises it.

### How are sensors prescribed?

Libre sensors last for 14 days.

Drug Tariff IXA says "FreeStyle Libre Sensor" (contains 1 sensor, 1 sensor applicator and 1 wipe) - £35

The Freestyle Libre reader is not available on prescription and is provided free of charge by the
company. 
        

### RMOC guidance

1. Patients who undertake intensive monitoring >8 times daily
2. Those who meet the current NICE criteria for insulin pump therapy (HbA1c >8.5% (69.4mmol/mol) or disabling hypoglycemia as described in NICE TA151) where a successful trial of FreeStyle Libre® may avoid the need for pump therapy.
3. Those who have recently developed impaired awareness of hypoglycaemia. It is
noted that for persistent hypoglycaemia unawareness, NICE recommend
continuous glucose monitoring with alarms and Freestyle Libre does currently
not have that function.
4. Frequent admissions (>2 per year) with DKA or hypoglycaemia.
5. Those who require third parties to carry out monitoring and where conventional
blood testing is not possible.



In [9]:
query = """
SELECT
  Quantity AS quantity,
  SUM(Items) AS items
FROM
  ebmdatalab.tmp_eu.raw_prescribing_data_2018_08
WHERE
  BNF_Code LIKE '{test_strip_code}'
GROUP BY
  Quantity

""".format(test_strip_code=test_strip_code)

test_strips = pd.read_gbq(query, GBQ_PROJECT_ID, dialect='standard', verbose=False)

/home/seb/.local/share/virtualenvs/jupyter/local/lib/python3.6/site-packages/pandas/io/gbq.py:108: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  **kwargs)


In [86]:
query = """SELECT
  month, SUM(quantity) AS quantity
FROM
  `ebmdatalab.hscic.normalised_prescribing_standard`
WHERE
  bnf_code = '{libre_code}'
GROUP BY month
ORDER BY month
  
""".format(libre_code=libre_code)
libres_by_month = pd.read_gbq(query, GBQ_PROJECT_ID, dialect='standard', verbose=False)

/home/seb/.local/share/virtualenvs/jupyter/local/lib/python3.6/site-packages/pandas/io/gbq.py:108: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  **kwargs)


In [75]:
libre_quantity = libres_by_month.iloc[-1]['quantity']/2
print("Assuming all Libre prescriptions are for Type 1 patients, about {}% of Type 1 patients in England have Libre prescribed".format(
    round((libre_quantity / type_one_in_england) * 100)))

Assuming all Libre prescriptions are for Type 1 patients, about 2.0% of Type 1 patients in England have Libre prescribed


In [85]:
per_day = 8
per_month = 8 * 31
high_prescription_items = test_strips[test_strips['quantity'] >= per_month]['items'].sum()
all_items = test_strips['items'].sum()
print("{}% of items in England are for high prescriptions meeting RMOC criteria.".format(
    round(high_prescription_items/(all_items + libres_by_month.iloc[-1]['quantity']/2) * 100)))
print("{all_items} items (~patients) in the most recent month".format(all_items=all_items))
type_2_and_other_glucose = all_items - type_one_in_england
print("If we assume all Type 1 are in that group, then that's {type_2_and_other_glucose} Type 2 patients and non-diabetics".format(type_2_and_other_glucose=type_2_and_other_glucose))
print("If every high prescription was for a Type 1 patient then that's at most {}% on high prescriptions".format(
    round(high_prescription_items/(type_one_in_england + libres_by_month.iloc[-1]['quantity']/2) * 100)))

3.0% of items in England are for high prescriptions meeting RMOC criteria.
597192 items (~patients) in the most recent month
If we assume all Type 1 are in that group, then that's 285552 Type 2 patients and non-diabetics
If every high prescription was for a Type 1 patient then that's at most 6.0% on high prescriptions


## Conclusions

* Approximately 2% of Type 1 patients are prescribed Libre
* At most 3% of *all* patients prescribed test strips meet RMOC criteria, based on test strip prescribing alone (criteron 1 above). If only Type 1 patients were prescribed at high levels, then at most 6% of Type 1 patients are on high test strip prescriptions or Libres. Where does the claim that 25% *should* be getting Libre come from?

### Questions
* Is test strip prescribing/use evenly distributed between people with Type 1 and Type 2? Or are Type 1 patients more likely to have to test more often? 
